Kaggle دریافت دیتاست از

In [ ]:
from google.colab import files
files.upload()  # آپلود فایل kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"reyhaneshirani","key":"d650f21d3a85952cae14e468381eeb6e"}'}

انتقال فایل و تنظیم مجوز ها

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

kaggle نصب و استفاده از ابزار

In [ ]:
!pip install kaggle
!kaggle datasets download -d naserabdullahalam/phishing-email-dataset
!unzip phishing-email-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset
License(s): CC-BY-SA-4.0
  0% 0.00/77.1M [00:00<?, ?B/s]
100% 77.1M/77.1M [00:00<00:00, 1.38GB/s]
Archive:  phishing-email-dataset.zip
  inflating: CEAS_08.csv             
  inflating: Enron.csv               
  inflating: Ling.csv                
  inflating: Nazario.csv             
  inflating: Nigerian_Fraud.csv      
  inflating: SpamAssasin.csv         
  inflating: phishing_email.csv      


---


نصب کتابخانه های مورد نیاز

In [ ]:

!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install pandas
!pip install matplotlib seaborn


---


بارگذاری و بررسی داده هاه

In [ ]:
import pandas as pd

df = pd.read_csv('phishing_email.csv')

# نمایش چند سطر ابتدایی
df.head()


,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


بررسی ستون ها و توزیع برچسب ها

In [ ]:
# نمایش اطلاعات دیتافریم
df.info()

# بررسی توزیع برچسب‌ها
df['label'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82486 entries, 0 to 82485
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_combined  82486 non-null  object
 1   label          82486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


,count
label,
1,42891
0,39595


---


پیش پردازش داده ها

In [ ]:
import pandas as pd

df = pd.read_csv('CEAS_08.csv')
texts = df['body']
labels = df['label']  # یا هر ستون مرتبط با برچسب‌ها

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, shuffle=True, random_state=42)

train_labels = train_labels.tolist()
val_labels = val_labels.tolist()
train_texts = train_texts.tolist()
val_texts = val_texts.tolist()

---


توکنایز کردن متن‌ها با BERT

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

---


ساخت دیتاست سفارشی PyTorch

In [ ]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

---


BERT آماده سازی مدل

In [ ]:
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

train_dataset = TextDataset(encodings, train_labels)
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

#from transformers import BertForSequenceClassification, AdamW
from torch.optim import AdamW

num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

---


آموزش مدل

In [ ]:
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

epochs = 5
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

for epoch in range(epochs):
    model.train()
    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
    print(f'Epoch {epoch + 1}/{epochs} completed.')

100%|██████████| 7831/7831 [14:53<00:00,  8.76it/s]


Epoch 1/5 completed.


100%|██████████| 7831/7831 [14:57<00:00,  8.73it/s]


Epoch 2/5 completed.


100%|██████████| 7831/7831 [14:57<00:00,  8.73it/s]


Epoch 3/5 completed.


100%|██████████| 7831/7831 [14:57<00:00,  8.73it/s]


Epoch 4/5 completed.


100%|██████████| 7831/7831 [14:56<00:00,  8.73it/s]

Epoch 5/5 completed.


---


ارزیابی مدل

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
val_dataset = TextDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

predictions = []
true_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

predicted_labels = label_encoder.inverse_transform(predictions)
true_labels_str = label_encoder.inverse_transform(true_labels)

accuracy = accuracy_score(true_labels_str, predicted_labels)
precision = precision_score(true_labels_str, predicted_labels, average='weighted')
recall = recall_score(true_labels_str, predicted_labels, average='weighted')
f1 = f1_score(true_labels_str, predicted_labels, average='weighted')

print(f'Validation Accuracy: {accuracy}')
print(f'Validation Precision: {precision}')
print(f'Validation Recall: {recall}')
print(f'Validation F1 Score: {f1}')

Validation Accuracy: 0.9784191035627634
Validation Precision: 0.9787303728290941
Validation Recall: 0.9784191035627634
Validation F1 Score: 0.9783817175643652


---


تابع پیش بینی برای متن جدید

In [ ]:
def predict_text(text):
    encoding = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors='pt')

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).cpu().numpy()[0]

    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]
    return predicted_label

---


تست پیش بیینی روی چند ایمیل دلخواه

In [ ]:
test_emails = [
    "We've noticed unusual activity in your account. Please verify your identity immediately.",
    "Lunch is scheduled for 1 PM today in the cafeteria. Don’t be late!"
]
for email in test_emails:
    print(f"📨 Email: {email}")
    print(f"🔍 Prediction: {predict_text(email)}\n")

📨 Email: We've noticed unusual activity in your account. Please verify your identity immediately.
🔍 Prediction: 1

📨 Email: Lunch is scheduled for 1 PM today in the cafeteria. Don’t be late!
🔍 Prediction: 1

